# Full method for $d=2$ (Algorithm 1)

This notebook contains a complete working example of Algorithm 1 from the paper 

<i>Sampling triangulations of manifolds using Monte Carlo methods</i> by <a href="https://www.maths.usyd.edu.au/u/ega/">Eduardo Altmann</a> and <a href="https://sites.google.com/view/jonathan-spreer/">Jonathan Spreer</a>.


## Triangulations basics

A $d$-dimensional manifold $M$ is a topological space that locally looks like Euclidean $d$-space. A triangulation $T$ of $M$ is a subdivision of $M$ into $d$-simplices glued along their $(d-1)$-dimensional face such that the underlying space $|T|$ of $T$ (the $d$-simplices factored by their gluings) is homeomorphic to $M$.

The *face-vector* or *f-vector* $f(T) = (f_0, f_1, f_2, .... , f_d)$ of $T$ stores the number of $i$-dimensional simplices $f_i$ in $T$ in all dimensions $0 \leq i \leq d$. 

Example: The boundary of the tetrahedron $\Delta$ has face vector $f(\Delta) = (4,6,4)$.

## Regina basics

In this file, $T$ is always a triangulation of a surface (or $2$-dimensional manifold) $S$, i.e., a set of triangles, identified along their edges. Data type is `regina.Triangulation2()`. Given a triagnulation `T`, type in `print T.detail()` to obtain an overview of what is stored in `T`, and print `T.fVector()` for its $f$-vector.

We can go from one triangulation `T` of $S$ to any other triangulation `T'` of $S$ by applying *Pachner moves* -- local modifications to `T` that change the isomorphism type of `T`, but not the topology of the underlying surface. Pachner moves in dimension two in regina are given by commands 

- `T.pachner(t)`: Stellar subdivision of triangle `t` into three triangles by placing a new vertex into the center of `t`. This move adds one vertex, three edges, and two triangles to `T` (it replaces `t` by three new triangles).
- `T.pachner(e)`: The *edge flip*. In the quadrilateral spanned by the two triangles of `T` containing edge `e`, remove `e` and replace it by the other diagonal `e'` in the quadrilateral. This move leaves the $f$-vector invariant.
- `T.pachner(v)`: Inverse of the stellar subdivision `T.pachner(t)`. This move removes vertex `v` to merge three triangles into one.

### Usage `c.pachner(face,check,perform)`

Rginas functions to perform moves are all organised the same way: a move always returns `True` or `False` on whether or not the operation was allowed / successful. It changes the underlying triangulation (if applicable), and takes as argument the face where the move is performed (the face that is going to be removed from the triangulation), and two booleans as optional arguments. The first one determining whether to check that the move is possible, the second one to determine whether to perform the move.

I.e., the command `T.pachner(v,True,False)`  takes as input a triangulation `T` and checks whether vertex `v` is contained in exactly three distinct triangles, returns `True` or `False` accordingly, but does not alter `T`.

In [1]:
# Imports

# for timing
import time
# for exponential function
import math
# for random choices
import random
# import all functions etc. from regina
from regina import *

### Generate a seed triangulation

This function produces a triangulation of the closed orientable surface of genus `g` with `n` triangles.

In [2]:
def surface(g,n):
    if n%2 != 0:
        print("number of triangles must be even")
        return None
    if n < 4*g-2:
        print("a triangulation of a genus",g,"closed orientable surface must have at least",4*g-2,"triangles")
        return None
    
    # create empty surface triangulation
    s = Triangulation2()
    
    # Build fundamental domain disk with 4g-2 triangles and boundary a 4g-gon
    #
    # add first triangle
    s.newSimplex()
    for i in range(4*g-3):
        # add next triangle
        s.newSimplex()
        # take last two triangles and...
        a = s.simplex(s.size()-1)
        b = s.simplex(s.size()-2)
        # glue them together (gluing face iterates, face 2 gets never glued)
        if i%2 == 0:
            a.join(0,b,Perm3())
        else:
            a.join(1,b,Perm3())
    
    # special case if we have a surface
    if g==0:
        s.newSimplex()
        a = s.simplex(0)
        b = s.simplex(1)
        a.join(2,b,Perm3())
        a.join(1,b,Perm3())
        a.join(0,b,Perm3())
    # special case if we have a torus
    elif g==1:
        a = s.simplex(0)
        b = s.simplex(1)
        a.join(2,b,Perm3(2,0,1))
        a.join(1,b,Perm3(1,2,0))
    # glue antipodal edges to each other to obtain genus g surface
    else:
        for k in range(2*g-2):
            a = s.simplex(k)
            b = s.simplex(k+2*g)
            a.join(2,b,Perm3(1,0,2))
        a = s.simplex(2*g-2)
        b = s.simplex(4*g-3)
        a.join(2,b,Perm3(2,0,1))        
        a = s.simplex(2*g-1)
        b = s.simplex(0)
        a.join(2,b,Perm3(2,0,1))   
        
    # add triangles as needed
    while s.size() < n:
        s.pachner(s.simplex(0),True,True)
    return s
#######################################################

### Propose next move 

This function computes probability to go up depending on state size `n` and parameter `gamma`$= 1/k$ for some natural number $k$. A return value of $i$ means an $i$-move is proposed next. 

In [3]:
#######################################################
def alpha(n,gamma):
    y= random.random()
    a = math.exp((-1)*gamma*n)
    # maybe to adjust later?
    b = 1
    if y < a:
        return 0
    elif y >= a and y < (b+a)/(b+1):
        return 1
    elif y >=(b+a)/(b+1):
        return 2
#######################################################

### Compute neighbouring triangulations

This function produces a dictionary of all `g`-neighbours (with isomorphism signatures as keys) of triangulation `iso` with f-vector `f`.

This function uses non-standard isomorphism signatures and hence requires `regina` version 7.3 or newer.

In [4]:
#######################################################
def neighbours(iso,f,g):
    nbrs = {}
    # going up (0-2-moves at every triangle)
    if g == 0:
        for n in range(f[2]):
            # create copy of tri in standard iso sig labelling
            target = Triangulation2.fromIsoSig(iso)
            # test if move is possible and if so, perform it
            if target.pachner(target.triangle(n), True, False):
                target.pachner(target.triangle(n), False, True)
                # get isomorphism signature of result, add it to neighbours
                tiso = target.isoSig_RidgeDegrees()
                # add edge needed to flip to obtain this neighbour (in standard iso sig labelling)
                if not tiso in nbrs:
                    nbrs[tiso]=n
        return nbrs
    # horizontally (1-1-move or edge flip at every edge contained in two triangles)
    elif g == 1:
        for e in range(f[1]):
            # create copy of tri in standard iso sig labelling
            target = Triangulation2.fromIsoSig(iso)
            # test if move is possible and if so, perform it
            if target.pachner(target.edge(e), True, False):
                target.pachner(target.edge(e), False, True)
                # get isomorphism signature of result, add it to neighbours
                tiso = target.isoSig_RidgeDegrees()
                # add edge needed to flip to obtain this neighbour (in standard iso sig labelling)
                if not tiso in nbrs:
                    nbrs[tiso]=e
        return nbrs
    # going doen (2-0-move at every vertex of degree three in three distinct triangles)
    if g == 2:
        for v in range(f[0]):
            # create copy of tri in standard iso sig labelling
            target = Triangulation2.fromIsoSig(iso)
            # test if move is possible and if so, perform it
            if target.pachner(target.vertex(v), True, False):
                target.pachner(target.vertex(v), False, True)
                # get isomorphism signature of result, add it to neighbours
                tiso = target.isoSig_RidgeDegrees()
                # add edge needed to flip to obtain this neighbour (in standard iso sig labelling)
                if not tiso in nbrs:
                    nbrs[tiso]=v
        return nbrs
#######################################################

### Choose move from proposal

This function takes a state triangulation given by isomorphism signature `iso` with f-vector `f` and paramter `gamma`. It computes a proposal, enumerates neighbours of `iso` and decides whether to perform the proposed move.

In [5]:
###############################################################################
def choosemove(iso, f, gamma):
    # a==0 go up, a==1 horizontal, a==2 go down
    a = alpha(f[2],gamma)
    ngbrs = neighbours(iso,f,a)
    # get number of neighbours of this type
    num_ngbrs = len(ngbrs.keys())
    # random number for proposal to move or stay
    i = random.random()
    # setup done
    
    # go up
    if a == 0:
        # stay where you are
        if i > float(num_ngbrs)/float(f[2]):
            return iso, f
        # go up (very likely)
        else:
            return random.choice(list(ngbrs.keys())), [f[0]+1,f[1]+3,f[2]+2]
    # edge flip
    if a == 1:
        # stay where you are
        if i > float(num_ngbrs)/float(f[1]):
            return iso, f
        # move sideways (very likely)
        else:
            return random.choice(list(ngbrs.keys())), f        
    # go down
    elif a == 2:
        # stay where you are
        # avoid division by zero
        if f[2] == 2:
            return iso, f
        if i > float(num_ngbrs)/float(f[2]-2):
            return iso, f
        # go down (unlikely)
        else:
            return random.choice(list(ngbrs.keys())), [f[0]-1,f[1]-3,f[2]-2]
###############################################################################

### Main function

This is the main function taking in see triangulation `iso` with f-vector `f`. It performs a random walk in the Pachner graph of length `steps` with parameter `gamma`. Parameter `verbose` decides print behaviour, `name` is the filename for the output file.

In [6]:
###############################################################################
def randomise(iso, f, steps, gamma, interval, offset, name):
    # initialise number of steps
    st = 0
    histogram = {}
    # save stuff to a file
    with open(name,"w") as fl:
        fl.write("")
    # open output file
    while st < steps + offset*interval-1:
        st += 1
        iso, f = choosemove(iso,f,gamma)
        if (interval != 0 and st % interval == 0 and st >= offset*interval):
            # open output file
            with open(name,"a") as fl:
                fl.write(iso+"\n")
            print("collecting triangulation",int((st-offset*interval)/interval+1),":",iso)
    return True
###############################################################################

### Test the method

A short test of our method.

In [7]:
# genus of orientable surface to sample triangulations from
genus = 2
# create minimal triangulation of surface
s = surface(genus,max(2,4*genus-2))
# starting iso sig
iso = s.isoSig_RidgeDegrees()
# starting f-vector
f = s.fVector()
# number of steps (not including offset)
steps = 10000
# interval (number of steps between samples)
interval = 100
# offset (number of omitted samples at beginning of sequence)
offset = 100
# paramter
gamma = 1/10
# collection interval
verb = 10
# file to store raw sampling data in
filename = "out_genus_"+str(genus)+"_gamma_"+str(gamma)+".txt"


ttt = time.time()
trigs = randomise(iso, f, steps, gamma, interval, offset, filename)
print("time spent:",time.time()-ttt)

collecting triangulation 1 : MwwLvvMvzzzzvvMLAPAPacfjmnqrtEBHDGFJGJLLfecfafccfaceeefeceff
collecting triangulation 2 : OwvMvAzvvLLLvvLAvLQPQaegiiktryzGBIKKHNNJMMfefccfacacefaaccdabbb
collecting triangulation 3 : OwvvvvALLMvvPwvvQMMAMaknlppquwwDDJIHFHJKMNcdcceeefffeefbbcffbaf
collecting triangulation 4 : MwwvvLvMzzvQzzvvAQLMackpnqrvwvuyDDIFGKJLcddabdebdffcffacfbbc
collecting triangulation 5 : KwLvvvvPLwvvMPzAMLQadqqmoAuDwCFCBEGJJIccccbcacdbeaabbfbbf
collecting triangulation 6 : KwvzLvLvLLwwAPzvPwQafhotvpvxxwyBEHHGIJcddaaabcccbcffeabbc
collecting triangulation 7 : EwLvLvvzLLvLQPAMadgqnsyzzABBxxCDccbafedaadaeffdb
collecting triangulation 8 : IwvLvvvPMzwwMwwzPMafqlospuwvyyzEGEFHccabeaaaaeffcaabff
collecting triangulation 9 : MwLvwwPzvvvvMPwvPvQQadhhmkmAzywxHEIGIJKLffdfabeabbbfdbafccfb
collecting triangulation 10 : KwLvvvzzvvALLPAwwPQadjquxzvxwyzEGFGHIJccbadedcbffeaacbfbf
collecting triangulation 11 : IwLLvzwwwvvvMQzvQMaegilnpwvBAzBGEFFHcacbdaaebebfaafcff
collecting triangulation 12 : KwLvv

In [8]:
trigs

True

In [9]:
iso

'gvLQdeffbaec'

In [10]:
f

[1, 9, 6]

In [11]:
s.fVector()

[1, 9, 6]